In [42]:
%%writefile streamlit_app.py

import json
import numpy as np
import random
import string
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import streamlit as st

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

# Load dataset
with open('Combined_training.json') as file:
    data = json.load(file)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Preprocess function
def preprocess(sentence):
    sentence = sentence.lower()
    sentence = ''.join([char for char in sentence if char not in string.punctuation])
    words = nltk.word_tokenize(sentence)
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Extract patterns and tags
patterns = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(preprocess(pattern))
        tags.append(intent['tag'])

# Encode the tags
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

# Vectorize the patterns using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(patterns)

# Define and train the Naive Bayes model
model = MultinomialNB()
model.fit(X, y)

# Chatbot response function
def get_response(user_input):
    user_input = preprocess(user_input)
    sequence = vectorizer.transform([user_input])
    prediction = model.predict(sequence)
    tag = label_encoder.inverse_transform(prediction)[0]
    
    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    
    return "I'm not sure how to respond to that. Can you please rephrase?"

# Streamlit app
st.markdown("<h1 style='text-align: center; color: orange;'>Academic Support Chatbot</h1>", unsafe_allow_html=True)

# Initialize session state for storing conversation history and feedback
if 'history' not in st.session_state:
    st.session_state.history = []
if 'feedback' not in st.session_state:
    st.session_state.feedback = []

# Input text box
user_input = st.text_input("You:", "")
if st.button("Send"):
    if user_input:
        response = get_response(user_input)
        st.session_state.history.append((user_input, response))
        st.session_state.feedback.append(None)  # Placeholder for feedback
        
        # Display the latest response
        st.write(f"Bot: {response}")

# Provide feedback for the latest response if available
if st.session_state.history and st.session_state.feedback[-1] is None:
    feedback_temp = st.radio("Was this response helpful?", ('Select an option', 'Yes', 'No'), key='current_feedback')

    # Submit Feedback button
    if feedback_temp != 'Select an option' and st.button("Submit Feedback"):
        st.session_state.feedback[-1] = feedback_temp
        st.write(f"Feedback submitted: {feedback_temp}")

# "View Feedback" Button
if st.button("View Feedback"):
    st.write("Feedback Summary:")
    for i, (user, bot) in enumerate(st.session_state.history):
        st.write(f"You: {user}")
        st.write(f"Bot: {bot}")
        feedback = st.session_state.feedback[i]
        if feedback is not None:
            st.write(f"Feedback: {feedback}")
        else:
            st.write("Feedback: No feedback given")
        st.write("---")

# Styling the Chatbot
st.markdown("""
    <style>
    .stTextInput {font-size: 18px;}
    .stButton button {background-color: #4CAF50; color: white;}
    .stButton button:hover {background-color: #45a049;}
    </style>
""", unsafe_allow_html=True)


Overwriting streamlit_app.py
